In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
np.random.seed(42)
tf.random.set_seed(42)

### Carregando o conjunto de dados para regressão. 

+ Vamos usar o conjunto de dados habitacional da Califórnia e criar um regressor com uma rede neural.

+ Depois de carregar os dados, dividimos em um conjunto de treinamento, um conjunto de validação e um conjunto de teste, e padronizamos todos os atributos.

In [3]:
# Baixa a base de dados.
housing = fetch_california_housing()

# Divide o conjunto total de exemplos em conjuntos de treinamento e teste.
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)

# Divide o conjunto de treinamento em conjuntos de treinamento (menor) e validação.
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

# Aplica padronização às matrizes de atributos.
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

### Criando e compilando o modelo.

+ Neste exemplos usaremos a API sequencial, mas o método `save()` funciona também para a API funcional.

In [5]:
model = keras.models.Sequential(
    [
        keras.layers.Dense(30, activation="relu", input_shape=[8]),
        keras.layers.Dense(30, activation="relu"),
        keras.layers.Dense(1)
    ]
)

model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))

### Treinando o modelo.

In [6]:
history = model.fit(X_train, y_train, 
                    epochs=10, 
                    validation_data=(X_valid, y_valid)
                   )

Epoch 1/10
363/363 [==============================] - 2s 4ms/step - loss: 1.8542 - val_loss: 1.3779
Epoch 2/10
363/363 [==============================] - 1s 3ms/step - loss: 0.7854 - val_loss: 0.6885
Epoch 3/10
363/363 [==============================] - 1s 3ms/step - loss: 0.6817 - val_loss: 0.6645
Epoch 4/10
363/363 [==============================] - 1s 3ms/step - loss: 0.6283 - val_loss: 0.5944
Epoch 5/10
363/363 [==============================] - 1s 3ms/step - loss: 0.5868 - val_loss: 0.5505
Epoch 6/10
363/363 [==============================] - 1s 4ms/step - loss: 0.5510 - val_loss: 0.5211
Epoch 7/10
363/363 [==============================] - 2s 5ms/step - loss: 0.5225 - val_loss: 0.5055
Epoch 8/10
363/363 [==============================] - 2s 5ms/step - loss: 0.4993 - val_loss: 0.4648
Epoch 9/10
363/363 [==============================] - 2s 5ms/step - loss: 0.4803 - val_loss: 0.4442
Epoch 10/10
363/363 [==============================] - 2s 6ms/step - loss: 0.4649 - val_loss: 0.4304

### Salvando o modelo.

In [8]:
model.save("./my_keras_model.h5")

### Carregando o modelo salvo.

In [9]:
loaded_model = keras.models.load_model("./my_keras_model.h5")

### Realizando predições com o modelo salvo.

In [11]:
X_new = X_test[:3]

y_pred = loaded_model.predict(X_new)

1/1 [==============================] - 0s 24ms/step


In [12]:
for i in range(len(y_pred)):
    print('Actual: %1.3f - Predicted: %1.3f' % (y_test[i], y_pred[i]))

Actual: 0.477 - Predicted: 0.804
Actual: 0.458 - Predicted: 1.559
Actual: 5.000 - Predicted: 3.003


### Salvando os pesos de modelos criados a partir de subclasses da classe `Model`.

+ O método `save()` e a função `load_model` funcionam ao usar as APIs sequencial e funcional, mas, infelizmente, não ao usar subclasses da classe `Model`. 

+ Porém, podemos usar os métodos `save_weights()` e `load_weights()` para pelo menos salvar e restaurar os parâmetros do modelo. 

+ Entretanto, precisaremos salvar e restaurar todo o resto nós mesmos.

In [14]:
model.save_weights("./my_keras_weights.ckpt")

#### Importante

Para carregar os pesos, **precisamos antes ter um modelo instanciado e compilado**.

In [18]:
model.load_weights("./my_keras_weights.ckpt")

Depois dos pesos terem sido carregados, podemos fazer predições normalmente.

In [16]:
y_pred = model.predict(X_new)

1/1 [==============================] - 0s 80ms/step


In [17]:
for i in range(len(y_pred)):
    print('Actual: %1.3f - Predicted: %1.3f' % (y_test[i], y_pred[i]))

Actual: 0.477 - Predicted: 0.804
Actual: 0.458 - Predicted: 1.559
Actual: 5.000 - Predicted: 3.003
